This notebook implements gradient descent for linear regression over a single feature.

First, we'll generate data points along some line, with random noise causing the points to be above or below the line.
Our goal will be to use linear regression to recover the original line, using only the (noisy) data points.

In [1]:
import random
original_w = 2.5
original_b = 20
x = []
y = []
for x_i in list(range(100)):
    noise = (random.random()-0.5)*10
    x.append(x_i)
    y.append(original_w * x_i + original_b + noise)

In [2]:
print(x[0:5])
print(y[0:5])

[0, 1, 2, 3, 4]
[15.630057646259187, 24.961768953166466, 29.86405035279268, 27.641313564634522, 29.076205576826563]


We'll need a cost function

In [3]:
def calculate_cost(x, y, w, b):
    cost = 0
    m = len(x)

    for i in range(m):
        f_wb = (w*x[i] + b)
        cost += (f_wb - y[i])**2

    cost = cost * (1 / (2*m))
    return cost

The cost won't be exactly zero, even with our original weight and bias, because we added noise to the data.

In [4]:
calculate_cost(x, y, original_w, original_b)

3.6850773669781196

But, we expect the cost to (usually) be lower than it is for other values of w and b:

In [5]:
print(calculate_cost(x,y, original_w + random.random(), original_b + random.random()*10))
print(calculate_cost(x,y, original_w + random.random(), original_b + random.random()*10))
print(calculate_cost(x,y, original_w + random.random(), original_b + random.random()*10))
print()
print(calculate_cost(x,y, original_w + random.random(), original_b))
print(calculate_cost(x,y, original_w + random.random(), original_b))
print(calculate_cost(x,y, original_w + random.random(), original_b))
print()
print(calculate_cost(x,y, original_w, original_b + random.random()*10))
print(calculate_cost(x,y, original_w, original_b + random.random()*10))
print(calculate_cost(x,y, original_w, original_b + random.random()*10))

386.048925937017
199.05152628165766
1052.895301430438

1121.3226067843816
70.27322266145595
750.5512795849261

3.8987296986933457
7.363463815970462
6.514978479913101


When we implement gradient descent, we'll initialize `w` and `b` to some random numbers and then iteratively update them. We'll update each in the direction that minimized the cost function `J`. In order to figure out how to change `w` and `b` to minimize the cost function, we need to know the partial of `J` w.r.t. `w` and the partial of `J` w.r.t. `b`. Together, we call these partials the gradient.

Here is a reminder about how to calculate the partial of `J` w.r.t. `w`:
- `J` is a function of `f`, `x` and `y`.
- So the partial of `J` w.r.t. `w` is: `(dJ/df)*(df/dw) + (dJ/dx)*(dx/dw) + (dJ/dy)*(dy/dw)`.
- The partials `(dx/dw)` and `(dy/dw)` are both equal to zero, so we can focus on the first part: `(dJ/df)*(df/dw)`.
- `J` is `(1/(2*m))*sum( (f(x_i) - y_i)**2 )` or (when we expand the squared term) `(1/(2*m))*sum( f(x_i)**2 - 2*f(x_i)*y_i + y_i**2 )`
- To make it easier to read, we might write this as:
- Then, `J = (1/2m)*sum( f**2 - 2fy + y**2 )`
- So `dJ/df` is `(1/2m)*sum( 2f - 2y + 0)` or `(1/m)*sum(f - y)`
- `f` is `wx + b`
- So `df/dw` is `x`.
- Putting this all together, we have:

```python
(dJ/dw)
    = (dJ/df)*(df/dw) + (dJ/dx)*(dx/dw) + (dJ/dy)*(dy/dw)
    = (dJ/df)*(df/dw) + 0               + 0
    = (dJ/df)*(df/dw)
    = ((1/m)*sum(f - y)) * x
    = ((1/m)*sum((f(x_i) - y_i)*x_i)
```

The derivation of `(dJ/db)` is the same, except that `(df/db)` is `1` (whereas `(df/dw)` was `x`).

In [6]:
def calculate_gradient(x,y,w,b):
    dJdw = 0
    dJdb = 0
    for i in range(len(x)):
        f = w*x[i] + b
        dJdw = (f - y[i])*x[i]
        dJdb = (f - y[i])
    dJdw = dJdw * (1.0/len(x))
    dJdb /= len(x)
    return dJdw, dJdb

In [7]:
calculate_gradient(x, y, original_w, original_b)

(3.1618381621009726, 0.03193775921314113)

Now that we can calculate gradients, we can implement gradient descent.

The way this will work is that we'll start with a random w and b, and then update them by the learning rate times their partial derivatives. There are many ways we might choose to stop, but let's try taking 100 steps and seeing how much we can minimize the cost function.

In [8]:
def gradient_descent(x, y, w_in, b_in, learn_rate, steps):
    w = w_in
    b = b_in
    for _ in range(steps):
        dJdw, dJdb = calculate_gradient(x,y,w,b)
        # print("dJdb is", dJdb)
        # print("dJdw is", dJdw)
        w -= learn_rate*dJdw
        b -= learn_rate*dJdb
    return w,b

Now, let's see if this process minimizes our cost function

In [9]:
random_w = (random.random()-0.5)*100
random_b = (random.random()-0.5)*100
print(f"random_w = {random_w:2f}\nrandom_b = {random_b:2f}")

random_w = -32.070543
random_b = 7.502585


In [10]:
w = random_w
b = random_b
print(f"initial_w = {w:2f}\ninitial_b = {b:2f}")
costs = []
for _ in range(100):
    w,b = gradient_descent(x,y,w,b,1e-5, 100)
    costs.append(calculate_cost(x,y,w,b))

print(f"final_w = {w:2f}\nfinal_b = {b:2f}")
print(f"goal_w = {original_w:2f}\ngoal_b = {original_b:2f}")
print(costs)


initial_w = -32.070543
initial_b = 7.502585
final_w = 2.588531
final_b = 7.852676
goal_w = 2.500000
goal_b = 20.000000
[1630819.9728555079, 1341156.9342931022, 1103010.0419689855, 907211.0826311165, 746224.0639972285, 613854.5760372402, 505010.9093758642, 415507.71340688574, 341904.6182796631, 281373.5941290614, 231589.92984084346, 190642.62507060665, 156960.73833912573, 129252.84971945937, 106457.30267181447, 87701.30549963967, 72267.31474539892, 59565.40380904253, 49110.5509989296, 40503.971025410494, 33417.76994586979, 27582.331786717805, 22775.950450416934, 18816.307130765304, 15553.464649707967, 12864.108641008896, 10646.813598052706, 8818.151330597642, 7309.491863708191, 6064.373514965031, 5036.340833721688, 4187.167125498669, 3485.393111602818, 2905.125460802729, 2425.0489465679325, 2027.6142164424782, 1698.3699272000633, 1425.413561580352, 1198.9398142037983, 1010.8691920623577, 854.5425637367225, 724.4699303337491, 616.123777991089, 525.7690871425472, 450.32348372728126, 387.2

The `w` and `b` we found isn't as good as our `original_w` and `original_b`:

In [11]:
calculate_cost(x, y, original_w, original_b)

3.6850773669781196

But it might perform better with additional training data. Let's give it 10000 points to work with:

In [12]:
original_w = 2.5
original_b = 20
x = []
y = []
for x_i in list(range(10000)):
    noise = (random.random()-0.5)*10
    x.append(x_i)
    y.append(original_w * x_i + original_b + noise)

In [13]:
# For a fair test, we'll use the same randomly chosen w and b from before.
w = random_w
b = random_b
print(f"initial_w = {w:2f}\ninitial_b = {b:2f}")
costs = []
for _ in range(100):
    w,b = gradient_descent(x,y,w,b,1e-5, 100)
    costs.append(calculate_cost(x,y,w,b))

print(f"final_w = {w:2f}\nfinal_b = {b:2f}")
print(f"goal_w = {original_w:2f}\ngoal_b = {original_b:2f}")
print(costs)

initial_w = -32.070543
initial_b = 7.502585
final_w = 2.501474
final_b = 7.506042
goal_w = 2.500000
goal_b = 20.000000
[52.9174710867869, 26.44129195286023, 26.44096286848585, 26.440962859752787, 26.440962859752787, 26.440962859752787, 26.440962859752787, 26.440962859752787, 26.440962859752787, 26.440962859752787, 26.440962859752787, 26.440962859752787, 26.440962859752787, 26.440962859752787, 26.440962859752787, 26.440962859752787, 26.440962859752787, 26.440962859752787, 26.440962859752787, 26.440962859752787, 26.440962859752787, 26.440962859752787, 26.440962859752787, 26.440962859752787, 26.440962859752787, 26.440962859752787, 26.440962859752787, 26.440962859752787, 26.440962859752787, 26.440962859752787, 26.440962859752787, 26.440962859752787, 26.440962859752787, 26.440962859752787, 26.440962859752787, 26.440962859752787, 26.440962859752787, 26.440962859752787, 26.440962859752787, 26.440962859752787, 26.440962859752787, 26.440962859752787, 26.440962859752787, 26.440962859752787, 26.4

Finally we'll do one more test with 100000 points.

In [14]:
original_w = 2.5
original_b = 20
x = []
y = []
for x_i in list(range(100000)):
    noise = (random.random()-0.5)*10
    x.append(x_i)
    y.append(original_w * x_i + original_b + noise)

In [15]:
# For a fair test, we'll use the same randomly chosen w and b from before.
w = random_w
b = random_b
print(f"initial_w = {w:2f}\ninitial_b = {b:2f}")
costs = []
for _ in range(100):
    w,b = gradient_descent(x,y,w,b,1e-5, 100)
    costs.append(calculate_cost(x,y,w,b))

print(f"final_w = {w:2f}\nfinal_b = {b:2f}")
print(f"goal_w = {original_w:2f}\ngoal_b = {original_b:2f}")
print(costs)

initial_w = -32.070543
initial_b = 7.502585
final_w = 2.500150
final_b = 7.502930
goal_w = 2.500000
goal_b = 20.000000
[25.965231357189246, 25.965231357189246, 25.965231357189246, 25.965231357189246, 25.965231357189246, 25.965231357189246, 25.965231357189246, 25.965231357189246, 25.965231357189246, 25.965231357189246, 25.965231357189246, 25.965231357189246, 25.965231357189246, 25.965231357189246, 25.965231357189246, 25.965231357189246, 25.965231357189246, 25.965231357189246, 25.965231357189246, 25.965231357189246, 25.965231357189246, 25.965231357189246, 25.965231357189246, 25.965231357189246, 25.965231357189246, 25.965231357189246, 25.965231357189246, 25.965231357189246, 25.965231357189246, 25.965231357189246, 25.965231357189246, 25.965231357189246, 25.965231357189246, 25.965231357189246, 25.965231357189246, 25.965231357189246, 25.965231357189246, 25.965231357189246, 25.965231357189246, 25.965231357189246, 25.965231357189246, 25.965231357189246, 25.965231357189246, 25.965231357189246, 